<a href="https://colab.research.google.com/github/NeuromarketingCientifico/vc_nm/blob/main/Funciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def seleccionar_archivo_colab():
    uploaded = files.upload()  # abre diálogo para subir archivo local
    if uploaded:
        nombre = list(uploaded.keys())[0]
        return nombre
    return None

In [ ]:
def seleccionar_archivo_colab():
    uploaded = files.upload()  # abre diálogo para subir archivo local
    if uploaded:
        nombre = list(uploaded.keys())[0]
        return nombre
    return None

In [ ]:
def mostrar_img(title, img):
    # cv usa BGR → convertir a RGB para matplotlib
    img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.imshow(img_rgb)
    plt.title(title)
    plt.axis("off")
    plt.show()

In [ ]:
def procesar_imagen_colab(ruta_imagen):
    nombre = os.path.basename(ruta_imagen)

    img = cv.imread(ruta_imagen)

    # ---------- SIFT + HEATMAP ----------
    sift = cv.SIFT_create()
    kp, _ = sift.detectAndCompute(img, None)
    num_puntos_sift = len(kp)

    heatmap = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)
    for kp_i in kp:
        x, y = int(kp_i.pt[0]), int(kp_i.pt[1])
        if 0 <= x < heatmap.shape[1] and 0 <= y < heatmap.shape[0]:
            heatmap[y, x] += 1

    heatmap = cv.GaussianBlur(heatmap, (25, 25), 0)
    heatmap = cv.normalize(heatmap, None, 0, 255, cv.NORM_MINMAX)
    heatmap_color = cv.applyColorMap(heatmap.astype(np.uint8), cv.COLORMAP_JET)
    combinado_sift = cv.addWeighted(img, 0.8, heatmap_color, 0.5, 0)

    mostrar_img("Resultado SIFT", combinado_sift)

    # ---------- YOLO ----------
    resultados_yolo = model.predict(source=img, conf=0.5, save=False)
    salida_yolo = img.copy()

    for r in resultados_yolo:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            etiqueta = r.names[int(box.cls[0])]
            cv.rectangle(salida_yolo, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv.putText(salida_yolo, etiqueta, (x1, y1 - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    mostrar_img("Resultado YOLO", salida_yolo)


    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    blurred = cv.GaussianBlur(gray, (21, 21), 0)
    mapa_saliencia = cv.normalize(blurred, None, 0, 255, cv.NORM_MINMAX)
    heatmap_sal = cv.applyColorMap(mapa_saliencia.astype(np.uint8), cv.COLORMAP_JET)
    combinado_yolo = cv.addWeighted(cv.cvtColor(img, cv.COLOR_BGR2RGB), 0.6, heatmap_sal, 0.4, 0)

    mostrar_img("Resultado YOLO + SIFT", combinado_yolo)
    # ---------- Cálculo de Área Saliente ----------
    umbral = 200
    area_saliente = int(np.sum(mapa_saliencia > umbral))
    total_pixeles = mapa_saliencia.size
    porcentaje = area_saliente / total_pixeles * 100


    # Guardar en lista de resultados
    resultados = []
    resultados.append({
        "Imagen": nombre,
        #"YOLO: Puntos": len(kp),
        "SIFT: Puntos": num_puntos_sift,
        "Área Saliente (pixeles)": int(area_saliente),
        "% Área Saliente": round(porcentaje, 2)
    })

    # Mostrar indicadores como DataFrame en pantalla
    df_resultados = pd.DataFrame(resultados)
    print(df_resultados)





